# Analysis of EMSight usage in telemetry data

## Setup

Please skip to **Filtering Data**

In [1]:
# setup
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from helper import *

In [2]:
# read in the data
df = pd.read_csv('../data/sdf_featured.csv')
data_range(df)

There are 78,682 sessions from 2016-09-15 to 2017-05-25


In [3]:
df.columns

Index(['session_id', 'buildnum', 'guid', 'instid', 'sess_user', 'company',
       'serial_num', 'created_time', 'has_commands', 'user_type', 'user_id',
       'disp_name', 'iu_name', 'custid', 'runtime', 'state', 'start_user',
       'proj_name', 'auto_proj', 'ele_created_n', 'commands_n', 'no_sim_b',
       'lin_sim_n', 'nonlin_sim_n', 'HB_legacy_b', 'HB_aplac_b', 'transient_b',
       'em_sim_b', 'load_pull_b', 'xmodels_b', 'ems_highcnt_b', 'ems_doc_b',
       'ems_sim_n', 'vss_doc_b', 'vss_sim_n', 'vss_rfi_b', 'vss_rfb_b',
       'vss_rfp_b', 'xem_sim_n', 'xem_doc_b', 'ana_sim_n', 'ana_doc_b',
       'tuning_b', 'optim_b', 'yield_b', 'extract_b', 'measurements_n',
       'lin_noise_b', 'osc_b', 'stability_b', 'cell_stretchers_b', 'antenna_b',
       'ifilter_b', 'symbol_gen_b', 'board_import_b', 'scripts_b',
       'script_ide_b', 'layout_b', 'gen_poly_b', 'pdk_b', 'layout_checks_b',
       'enable_disable_b', 'markers_b', 'freeze_traces_b', 'schematic_eq_b',
       'output_eq_b', '

## Filtering Data

Not all data is important in performing this analysis.  We will exclude:

* Data from internal users
* Simulations using our examples that use EMSight
* Sessions that do not have EMSight simulations

In [5]:
# filter by build, external, emsight as applicable
excl_beta = df.buildnum==8316
external = df.user_type != 'internal'
emsight = ((df.ems_doc_b == 1 ) | (df.ems_sim_n > 0))
excl_ex = df.auto_proj != False

ext = df[external & excl_ex]
ems = df[external & emsight & excl_ex]  # change emsight to emsight & excl_beta to see only production builds
print('Number of sessions with an EMSight document or simulation is {} out of {} external sessions.'.format(len(ems), len(ext)))

Number of sessions with an EMSight document or simulation is 791 out of 73863 external sessions.


## Users

Rather than counting on the username produced in the start table, the telemetry system tries to identify unique users by combining other information such as machine and company to combine users with multiple user names.  This gives a more reliable user count.

In [6]:
# Let's look at each user that's used emsight by session count (those > 5 sessions)
large_users = ems.user_id.value_counts()
print('cnt  Company, User')
for user_id, count in large_users[large_users > 5].iteritems():
    n, c = get_name_and_company(ems, user_id)
    print('{:3d} - {}, {}'.format(count, c, n))

cnt  Company, User
 75 - PoweRFul Microwave, Dominic Fitzpatric at PoweRFul
 64 - AMCOM Communications, Amin Ezzeddine @ AMCOM
 58 - Gigacom, Inc., Randy Bell
 48 - AGH University of Science and Technology, Ilona Piekarz
 29 - Wide Band Systems, Inc., Mark Stanley
 24 - IBM Corp - TJ Watson Research Center, Salvatore Olivandese @ IBM
 24 - Filtronic Broadband Ltd., Martin Holroyd @ Filtronic
 21 - AGH University of Science and Technology, Jakub Sorocki
 20 - University of Colorado at Boulder - Department of Electrical Engineering, Maxime Malnou
 19 - Mercury Systems, Inc. - Corporate Headquarters, Jeff Miner @ Mercury
 17 - Space Exploration Technologies Corporation (SpaceX), Daniel Meinzer at SpaceX
 16 - Friends, nan
 16 - University of Guanajuato, Gustavo Vallejo
 14 - AR RF/Microwave Instrumentation, Gene Akin @ AR
 13 - Virginia Tech, Kanika Saini
 13 - Ecole SupÃ©rieure des Technologies Electronique Informatique Infographie (ESTEI), LAPORTE Martin
 12 - Cree, Inc. - Research Tria

Unfortunately this only tells part of the story because EMSight is also used by XModels.  Let's if these users also use XModels and Axiem.

In [8]:
for uid, grp_df in ems.groupby('user_id'):
    if len(grp_df) > 5:
        n, c = get_name_and_company(ems, uid)
        print('{} sessions for "{}" - {}'.format(len(grp_df), c, n))
        for col in ['ems_sim_n', 'ems_doc_b', 'xmodels_b', 'ems_highcnt_b', 'xem_doc_b', 'xem_sim_n']:
            print('    {} - {}'.format(col, grp_df[col].sum()))
    

75 sessions for "PoweRFul Microwave" - Dominic Fitzpatric at PoweRFul
    ems_sim_n - 2719.0
    ems_doc_b - 0.0
    xmodels_b - 75.0
    ems_highcnt_b - 4.0
    xem_doc_b - 0.0
    xem_sim_n - 4190.0
17 sessions for "Space Exploration Technologies Corporation (SpaceX)" - Daniel Meinzer at SpaceX
    ems_sim_n - 160.0
    ems_doc_b - 1.0
    xmodels_b - 13.0
    ems_highcnt_b - 0.0
    xem_doc_b - 0.0
    xem_sim_n - 0.0
19 sessions for "Mercury Systems, Inc. - Corporate Headquarters" - Jeff Miner @ Mercury
    ems_sim_n - 2100.0
    ems_doc_b - 1.0
    xmodels_b - 19.0
    ems_highcnt_b - 1.0
    xem_doc_b - 0.0
    xem_sim_n - 0.0
29 sessions for "Wide Band Systems, Inc." - Mark Stanley
    ems_sim_n - 101.0
    ems_doc_b - 0.0
    xmodels_b - 0.0
    ems_highcnt_b - 0.0
    xem_doc_b - 2.0
    xem_sim_n - 9.0
10 sessions for "Agile Microwave Technology Inc" - nan
    ems_sim_n - 20.0
    ems_doc_b - 0.0
    xmodels_b - 9.0
    ems_highcnt_b - 0.0
    xem_doc_b - 0.0
    xem_sim_n - 

## Conclusion on EMSight Use

**CAVEAT**: we have to remember that only 40% of customer sessions are sending telemetry so this is an incomplete analysis.  It is not clear whether we can just scale these results to the wider audience.

Looking at the data, 2 of these users, Dominic and Salvatore) show all sessions containing XModels and several sessions with a large number of EMSight simulations (>50) which is very indicative of XModel use rather than using EMSight as their EM simulator.  These users also show many Axiem simulations which is re-enforces the idea that EMSight is just being used for Xmodels.  In digging into it, Meinzer at SpaceX is also only using EMSight for XModels.

Agile microwave is an interesting case.  They have 10 sessions, all with exactly 2 simulations in EMSight.

Mark Stanley at Wide Band is the one user that is definitely using EMSight.



## XModels

Just to close the loop let us look at the number of sessions and users that use XModels, regardless of whether they have used EMSight to fill tables.

In [11]:
ext.xmodels_b.sum()

7624.0

In [12]:
print('Out of {:,} external sessions, {:,.0f} ({:2.0f}%) of them contain XModels'.format(len(ext), 
                                                                                  ext.xmodels_b.sum(), 
                                                                                  100*ext.xmodels_b.sum()/len(ext)))

Out of 73,863 external sessions, 7,624 (10%) of them contain XModels


Let's take a look at the biggest XModel users.  This list is the number of sessions per user that use XModels.  This only shows those with more than 20 sessions.  Note, sessions using our examples is excluded in this.

In [14]:
xmod = ext[ext.xmodels_b  == 1]
large_users = xmod.user_id.value_counts()
print('cnt  Company, User')
for user_id, count in large_users[large_users > 20].iteritems():
    n, c = get_name_and_company(xmod, user_id)
    print('{:3d} - {}, {}'.format(count, c, n))

cnt  Company, User
334 - Cobham Sensor Systems - TX, Robert Flynt @ Cobham
283 - Qorvo, Inc. - TX Government, Gary Burgin at Qorvo
278 - Mitsubishi Electric Co., Ltd. - HF & Optical Device Works, HF Device Department, Kanaya Ko at Mitsubishi
216 - Ivan Boshnakov, Ivan Boshnakov Consultant
211 - Cobham SATCOM, nan
201 - Advanced Semiconductor Business, Inc. (ASB), user6 @ ASB
199 - Cobham, Morten Hagensen, Ioannis Nissopoulos
184 - Analog Devices - Chelmsford, nan
173 - Friends, nan
140 - AMCOM Communications, Amin Ezzeddine @ AMCOM
136 - PoweRFul Microwave, Dominic Fitzpatric at PoweRFul
128 - Qorvo, Inc. - TX IDP, csanabria @ Qorvo TX
125 - MACOM Technology Solutions - AU, MAU59429 @ MA-COM
113 - (unknown), Suresh Ojha
 98 - Skyware Global, Andrew McGregor
 97 - Advanced Semiconductor Business, Inc. (ASB), user5 @ ASB
 94 - Mercury Systems, Inc. - Corporate Headquarters, Jeff Miner @ Mercury
 90 - Active Wave Technology, David Farkas @ Active Wave
 84 - Ericsson AB - Kista (WWserver),